# Tracking the nuclei

In [51]:
import os
import numpy as np
import pandas as pd
import glob
from pathlib import Path
from skimage.io import imread
from tracker.core import track
from tqdm import tqdm
import trackpy as tp
import matplotlib.pyplot as plt

Loading file paths

In [55]:
root = "/home/jack/workspace/own/cell-tracker"

data_path = os.path.join(root, "data/")
save_path = os.path.join(root, "output/")

data_path = Path(data_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

files = sorted(data_path.glob("*-Hoechst.tif"))

In [56]:
print(files)

[PosixPath('/home/jack/workspace/own/cell-tracker/data/001-1-Hoechst.tif'), PosixPath('/home/jack/workspace/own/cell-tracker/data/001-2-Hoechst.tif')]


Testing to choose the best parameters

In [ ]:
# Change the frame number, the diameter, and the separation value to check that the nuclei are recognized.
# Set topn to the number of cells to detect.

frames = imread(os.path.abspath(files[1]))
print(len(frames))
frame_no = 20
# f = tp.locate(frames[frame_no], diameter=25, minmass=0, topn=3, separation=7)
f = tp.locate(frames[frame_no], diameter=25, topn=3, separation=7)
print(f)
# tp.annotate(f, frames[frame_no])
# # plt.hist(f['mass'])
# plt.show()


131
            y          x           mass  ...       signal   raw_mass  ep
38  50.628920  46.571182  134107.150603  ...   976.941693  1056240.0 NaN
21  29.766445  43.497690  159922.516744  ...   994.839008  1132508.0 NaN
26  39.119467  84.934115  323617.367925  ...  1832.657045  1462765.0 NaN

[3 rows x 8 columns]


Running trackpy on all files

In [47]:
# Parameters

## Tracking parameters
diameter = 25  #31 is better for 1 cell
separation = 7
topn = 3 # Number of cell
verbose = True

## Saving parameters
start_position = 1
stop_position = 1
overwrite = False
old_suffix = '-Hoechst.tif'
new_suffix = '-tracking.csv'

In [48]:
files = glob.glob(os.path.join(data_path, "*-Hoechst.tif"))

for filename in tqdm(files):
    print(filename)
    print(os.path.basename(filename)[:3])
    position = int(os.path.basename(filename)[:3])
    if position>=start_position and position<=stop_position:

        frames = imread(os.path.abspath(filename))

        t = track(frames, diameter=diameter, separation=separation, 
                           topn=topn, verbose=verbose)

        savename = os.path.basename(filename).removesuffix(old_suffix) + new_suffix
        savename = os.path.join(save_path, savename)
        if os.path.exists(savename):
            if overwrite:
                print(f"Warning: the file {savename} already exists and will be deleted.")
                os.remove(savename)
                t.to_csv(savename)
            else:
                print(f"Warning: the file {savename} already exists, the data will not be saved.")
        else:
            t.to_csv(savename)

100%|██████████| 2/2 [00:01<00:00,  1.26it/s]

Frame 130: 3 trajectories present.
Tracking of nuclei completed.
